In [5]:
# -------------------------------------------------------
# AI Research Assistant Agent - Free Version (No API Key)
# -------------------------------------------------------

# Install dependencies (only once)
!pip install transformers

# Imports
import requests
from xml.etree import ElementTree as ET
from transformers import pipeline

# Load summarization model (DistilBART from Hugging Face)
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# --------------------------
# Tool 1: arXiv Search
# --------------------------
def arxiv_search_results(query, max_results=3):
    if not query.strip():
        return "Query was empty. Please enter a valid topic."

    base_url = "http://export.arxiv.org/api/query"
    params = {
        "search_query": f"all:{query}",
        "start": 0,
        "max_results": max_results
    }

    response = requests.get(base_url, params=params)
    entries = []

    if response.status_code == 200:
        root = ET.fromstring(response.text)
        for entry in root.findall("{http://www.w3.org/2005/Atom}entry"):
            title = entry.find("{http://www.w3.org/2005/Atom}title").text.strip()
            summary = entry.find("{http://www.w3.org/2005/Atom}summary").text.strip()
            entries.append(f"{title}: {summary}")
    else:
        entries.append("Error: Unable to connect to arXiv API. Try again later.")

    return "\n\n".join(entries)

# --------------------------
# Agent Reasoning + Output
# --------------------------
def summarize_research(query):
    print("\n[Step 1] Fetching articles...")
    articles = arxiv_search_results(query)

    print("\n[Step 2] Generating summary...")

    # Chain-of-Thought-style prompt summary
    prompt = f"""
    Below are a few academic article abstracts about: "{query}"

    {articles}

    Please summarize the overall topic clearly. Then list three bullet points highlighting key findings.
    Close with a suggestion for further reading.
    """

    result = summarizer(prompt, max_length=180, min_length=50, do_sample=False)

    structured_output = f"""
    ====== Research Summary ======

    Topic: {query}

    {result[0]['summary_text']}

    =================================
    """
    return structured_output

# --------------------------
# Feedback Mechanism (RL Sim)
# --------------------------
def get_user_feedback():
    feedback = input("Was this summary helpful? (yes/no): ").strip().lower()
    if feedback == "yes":
        print("Great! Your feedback helps improve the agent.")
    elif feedback == "no":
        print("Thanks for the feedback. We’ll refine the output.")
    else:
        print("Invalid input. Feedback not recorded.")

# --------------------------
# Run the Agent
# --------------------------
print("Welcome to the AI Research Assistant Agent")
query = input("Enter your research topic: ").strip()

if not query:
    print("No topic entered. Please restart and try again.")
else:
    print(summarize_research(query))
    get_user_feedback()


Device set to use cpu


Welcome to the AI Research Assistant Agent
Enter your research topic: A.i. in healthcare

[Step 1] Fetching articles...

[Step 2] Generating summary...

    ====== Research Summary ======

    Topic: A.i. in healthcare

     The integration of artificial intelligence (AI) and machine learning (ML) into healthcare systems holds great promise for enhancing patient care and care-delivery efficiency . However, it also exposes sensitive data and system integrity to potential cyberattacks . This paper takes a thorough examination of existing healthcare AI S&P research, providing a unified framework that allows the identification of under-explored areas .

    
Was this summary helpful? (yes/no): Yes
Great! Your feedback helps improve the agent.
